In [27]:
import ast
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

from copy import deepcopy
from sklearn.model_selection import KFold

In [28]:
class MLP_500_500_500_classifier(nn.Module):
    def __init__(self, input_len):
        super().__init__()

        self.layer1 = nn.Linear(input_len, 500)  
        self.relu = nn.ReLU()                          
        self.hidden_layer_relu = nn.Sequential(
            nn.Linear(500,500),
            nn.ReLU(),
            nn.Linear(500,500),
            nn.ReLU(),
            nn.Linear(500,500),
            nn.ReLU()
        )
        self.layer2 = nn.Linear(500, 30)

    def forward(self, x):

        x = self.relu(self.layer1(x))
        x = self.hidden_layer_relu(x)
        logits = self.layer2(x)
        
        return logits

In [29]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

y_best_ansatz = np.argmax(y, axis=1)  

# criando o kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [31]:
# implementação de um critério de parada para o modelo parar no ponto "ótimo"
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [32]:
# 3. Função para treinar um fold
def train_fold(model, train_loader, val_loader, criterion, optimizer, n_epochs=1000000, patience = 10, min_delta = 0.0):
    model.to(device)
    early_stopping = EarlyStopping(patience=patience, min_delta=min_delta)
    best_model = None
    best_acc = 0.0
    
    for epoch in range(n_epochs):
        # Treinamento
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.squeeze()  # Converte (batch_size, 1) para (batch_size,)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validação
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            correct = 0
            total = 0
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                y_val = y_val.squeeze()  # Garante que y_val é 1D
                
                outputs = model(X_val)
                val_loss += criterion(outputs, y_val).item()  
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == y_val).sum().item()
                total += y_val.size(0)
        
        
        val_acc = correct / total
        val_loss /= len(val_loader)
        train_loss /= len(train_loader)
        
        print(f'Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.4f}')
        
        # Early Stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
        # Salvar melhor modelo
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = deepcopy(model.state_dict())
    
    return best_model, best_acc

In [33]:
results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n=== Fold {fold + 1}/{3} ===")
    
    # Dividir e preparar dados
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_best_ansatz[train_idx], y_best_ansatz[val_idx]
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    
    # Inicializar modelo e otimizador
    model = MLP_500_500_500_classifier(22)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # L2 regularization
    
    # Treinar fold
    best_model, best_acc = train_fold(model, train_loader, val_loader, criterion, optimizer, patience=15)
    results.append(best_acc)
    
    # Salvar modelo se necessário
    torch.save(best_model, f'./models_salvos/best_model_MLP-500-500-500-classifier_fold{fold+1}.pt')

# 7. Resultados finais
print("\n=== Resultados ===")
print(f"Acurácia média: {np.mean(results):.4f} ± {np.std(results):.4f}")
print(f"Acurácias por fold: {results}")


=== Fold 1/3 ===
Epoch 1/1000000 - Train Loss: 3.1483 - Val Loss: 3.2008 - Val Acc: 0.3000
Epoch 2/1000000 - Train Loss: 2.7183 - Val Loss: 2.8224 - Val Acc: 0.3000
Epoch 3/1000000 - Train Loss: 2.4985 - Val Loss: 2.7137 - Val Acc: 0.2583
Epoch 4/1000000 - Train Loss: 2.3608 - Val Loss: 2.7547 - Val Acc: 0.2583
Epoch 5/1000000 - Train Loss: 2.2703 - Val Loss: 2.7003 - Val Acc: 0.2583
Epoch 6/1000000 - Train Loss: 2.2668 - Val Loss: 2.7410 - Val Acc: 0.2583
Epoch 7/1000000 - Train Loss: 2.2705 - Val Loss: 2.6900 - Val Acc: 0.2583
Epoch 8/1000000 - Train Loss: 2.2368 - Val Loss: 2.6717 - Val Acc: 0.2583
Epoch 9/1000000 - Train Loss: 2.2528 - Val Loss: 2.6982 - Val Acc: 0.2667
Epoch 10/1000000 - Train Loss: 2.2007 - Val Loss: 2.6918 - Val Acc: 0.2667
Epoch 11/1000000 - Train Loss: 2.2030 - Val Loss: 2.6928 - Val Acc: 0.3000
Epoch 12/1000000 - Train Loss: 2.1722 - Val Loss: 2.7194 - Val Acc: 0.2750
Epoch 13/1000000 - Train Loss: 2.1966 - Val Loss: 2.7307 - Val Acc: 0.2833
Epoch 14/1000000

In [34]:
model = MLP_500_500_500_classifier(22).to(device)
#carregando o melhor fold treinado
model.load_state_dict(torch.load('.//models_salvos/best_model_MLP-500-500-500-classifier_fold2.pt'))  # Substitua pelo caminho correto
model.eval()  # Modo de avaliação

MLP_500_500_500_classifier(
  (layer1): Linear(in_features=22, out_features=500, bias=True)
  (relu): ReLU()
  (hidden_layer_relu): Sequential(
    (0): Linear(in_features=500, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=500, bias=True)
    (5): ReLU()
  )
  (layer2): Linear(in_features=500, out_features=30, bias=True)
)

In [35]:
input = X[6]
input_tensor = torch.FloatTensor(input).to(device)

with torch.no_grad():
    logits = model(input_tensor)
    print(logits)
    print(logits.shape)
    probabilities = torch.softmax(logits, dim=0)  # Shape (1, 30)

tensor([ 3.0814, -0.5331,  1.8702, -2.4175, -7.4428, -7.7259,  2.3016,  2.8937,
        -0.8710, -4.5846, -2.9946,  1.4828,  1.7127,  1.5595,  2.0949, -7.3915,
        -0.1745, -2.4360, -5.3278, -5.5369, -4.2518,  1.5561, -3.6381, -4.4087,
        -3.5264, -0.4362, -5.5552, -4.5998, -5.5612, -2.2605])
torch.Size([30])


In [36]:
print(probabilities)

tensor([2.5091e-01, 6.7570e-03, 7.4731e-02, 1.0264e-03, 6.7438e-06, 5.0808e-06,
        1.1504e-01, 2.0796e-01, 4.8194e-03, 1.1754e-04, 5.7638e-04, 5.0725e-02,
        6.3835e-02, 5.4770e-02, 9.3552e-02, 7.0986e-06, 9.6714e-03, 1.0077e-03,
        5.5902e-05, 4.5355e-05, 1.6396e-04, 5.4584e-02, 3.0286e-04, 1.4015e-04,
        3.3865e-04, 7.4446e-03, 4.4530e-05, 1.1577e-04, 4.4266e-05, 1.2010e-03])


In [37]:
# Obter índices e valores das top-k probabilidades (ex.: top-5)
k = 3
top_k_probs, top_k_indices = torch.topk(probabilities, k=k, dim=0)

# Converter para numpy e exibir
top_k_probs = top_k_probs.cpu().numpy().flatten()
top_k_indices = top_k_indices.cpu().numpy().flatten()

print("Melhores ansatzes e suas probabilidades:")
for i, (idx, prob) in enumerate(zip(top_k_indices, top_k_probs)):
    print(f"{i+1}º: Ansatz {idx} - Probabilidade: {prob:.4f}")

Melhores ansatzes e suas probabilidades:
1º: Ansatz 0 - Probabilidade: 0.2509
2º: Ansatz 7 - Probabilidade: 0.2080
3º: Ansatz 6 - Probabilidade: 0.1150
